In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

In [9]:


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]


In [10]:
class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)
    
FIFOQueue = deque

LIFOQueue = list

In [11]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

def g(n): return n.path_cost


def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        #print(self.map.neighbors[state])#añadido para ver
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))
        
        
        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result 

In [ ]:
#Exercise 3.7

Problem = Map(
    {('S', 'V1'):  1, ('S', 'V4'): 1, ('S', 'V8'): 1, ('V1', 'V5'): 1, ('V1', 'V11'): 1, 
     ('V4', 'V3'): 1, ('V5', 'V12'):  1, ('V8', 'V7'): 1, ('V7', 'V15'): 1, ('V12', 'V16'): 1, 
     ('V16', 'V15'): 1, ('V15', 'V23'):  1, ('V11', 'V14'): 1, ('V11', 'V17'): 1, ('V14', 'V23'): 1, 
     ('V23', 'V22'):  1, ('V22', 'V32'):  1, ('V17', 'V19'): 1, ('V19', 'V29'):  1, ('V3', 'V24'): 1, 
     ('V24', 'V29'):  1, ('V3', 'V26'):  1, ('V26', 'V27'): 1, ('V3', 'V26'):  1, ('V26', 'V27'): 1, 
     ('V27', 'V30'):  1, ('V30', 'V29'): 1, ('V29', 'V33'):  1, ('V33', 'V32'): 1, ('V32', 'G'): 1, ('V30', 'G'): 1})

travel = RouteProblem('S', 'G', map=Problem)

print("Depth First Search Answer:")
path_states(breadth_first_search(travel))


Depth First Search Answer:


['S', 'V4', 'V3', 'V24', 'V29', 'V30', 'G']

In [12]:

#Exercise 3.9

def never_outnumbered(missionaries_right, cannibals_right):
    """Verify if missionaries are not outnumbered) by cannibals"""
    missionaries_left = 3 - missionaries_right
    cannibals_left = 3 - cannibals_right

    # Missionaries must not be outnumbered on either side
    if (missionaries_right < cannibals_right and missionaries_right > 0) or (missionaries_left < cannibals_left and missionaries_left > 0):
        return False
    return True

def dfs_algorithm(state, path, visited):
    """Perform DFS with a predefined move order."""
    missionaries, cannibals, boat = state

    # If goal is reached
    if state == (0, 0, 0):
        print("\nSolution found!")
        for step in path:
            print(step)
        return True

    # Predefined move sequence
    moves = [
        (0, 2, "Take 2 Cannibals"),
        (0, 1, "Leave 1 Cannibal, Return with 1"),  
        (1, 1, "Take 1 Missionary Across"), 
        (0, 1, "Return 1 cannibal, take 1 Missionary"),  
        (2, 1, "Take 2 Missionaries Across"),  
        (1, 0, "Take 1 Cannibal Back"),  
        (0, 2, "Take 1 Cannibals an 1 missionary Across"), 
        (1, 0, "Return with 1 Canniba;"),  
        (2, 0, "Take 2 Cannibals Across")  
    ]

    for m, c, action in moves:
        if boat == 1:  # Boat on right side
            new_state = (missionaries - m, cannibals - c, 0)
        else:  # Boat on left side
            new_state = (missionaries + m, cannibals + c, 1)

        # Check if the move is valid and state not visited
        if (0 <= new_state[0] <= 3 and 0 <= new_state[1] <= 3 and
            never_outnumbered(new_state[0], new_state[1]) and
            new_state not in visited):

            visited.add(new_state)
            print(f"Move: {action} -> New State: {new_state}")

            if dfs_algorithm(new_state, path + [(action, new_state)], visited):
                return True

            visited.remove(new_state)  

    return False  # No solution from this path

# Start DFS
initial_state = (3, 3, 1)
visited_states = {initial_state}
dfs_algorithm(initial_state, [("Start", initial_state)], visited_states)

        


Move: Take 2 Cannibals -> New State: (3, 1, 0)
Move: Leave 1 Cannibal, Return with 1 -> New State: (3, 2, 1)
Move: Take 2 Cannibals -> New State: (3, 0, 0)
Move: Leave 1 Cannibal, Return with 1 -> New State: (3, 1, 1)
Move: Take 2 Cannibals Across -> New State: (1, 1, 0)
Move: Take 1 Missionary Across -> New State: (2, 2, 1)
Move: Take 2 Missionaries Across -> New State: (0, 1, 0)
Move: Take 2 Cannibals -> New State: (0, 3, 1)
Move: Leave 1 Cannibal, Return with 1 -> New State: (0, 2, 0)
Move: Return 1 cannibal, take 1 Missionary -> New State: (0, 2, 0)
Move: Leave 1 Cannibal, Return with 1 -> New State: (0, 2, 1)
Move: Take 2 Cannibals -> New State: (0, 0, 0)

Solution found!
('Start', (3, 3, 1))
('Take 2 Cannibals', (3, 1, 0))
('Leave 1 Cannibal, Return with 1', (3, 2, 1))
('Take 2 Cannibals', (3, 0, 0))
('Leave 1 Cannibal, Return with 1', (3, 1, 1))
('Take 2 Cannibals Across', (1, 1, 0))
('Take 1 Missionary Across', (2, 2, 1))
('Take 2 Missionaries Across', (0, 1, 0))
('Leave 1 Cann

True